## Strategic Divergence Analysis Summary

The strategic divergence visualizations reveal critical blind spots in the 80% ESPN + 20% ADP probability system:

### Key Findings:

1. **Queue Jump Alerts**: Players like Rachaad White (ESPN 85, ADP 27) may be drafted much earlier than the 80/20 system predicts

2. **Hidden Gems**: Players like Terry McLaurin and D'Andre Swift show massive positive divergence, suggesting significant value opportunities

3. **Overvalued Traps**: Veterans like Stefon Diggs and Keenan Allen rank much higher in ESPN than ADP, indicating potential reaches

4. **Position Patterns**: RBs show highest average positive divergence (+11.2), suggesting ESPN consistently undervalues RB depth

### Strategic Recommendations:

- **Immediate Targets**: Focus on high divergence players with good availability (Terry McLaurin, D'Andre Swift)
- **Urgent Picks**: Don't let Rachaad White slip past early picks - major divergence with low availability  
- **Avoid Reaches**: Be cautious about ESPN favorites like Stefon Diggs who ADP suggests are overvalued
- **Position Strategy**: Look for RB value in middle rounds where ESPN divergence is highest

These visualizations help identify when to override the 80/20 system and capitalize on ranking inefficiencies.

In [1]:
def plot_blind_spot_radar(df, min_divergence=15):
    """Radial plot of undervalued players (ADP much better than ESPN)"""
    
    # Find players where ADP ranks them much higher (ESPN rank - ADP rank > min_divergence)
    blind_spots = df[df['rank_divergence'] > min_divergence].copy()
    blind_spots = blind_spots.sort_values('abs_divergence', ascending=False).head(8)
    
    if len(blind_spots) == 0:
        print(f"No players found with divergence > {min_divergence}")
        return None
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatterpolar(
        r=blind_spots['abs_divergence'],
        theta=blind_spots['player_name'],
        mode='markers+lines',
        marker=dict(
            size=blind_spots['vbd_score']/3,  # Size by VBD score
            color=blind_spots['probability_available'],
            colorscale='RdYlGn',
            showscale=True,
            colorbar=dict(title="Availability<br>Probability")
        ),
        text=blind_spots['player_name'],
        hovertemplate='Player: %{text}<br>Divergence: %{r}<br>Availability: %{marker.color:.2f}<extra></extra>',
        fill='toself',
        opacity=0.6
    ))
    
    fig.update_layout(
        title=f'Blind Spots: ADP >{min_divergence} ranks better than ESPN',
        polar=dict(
            radialaxis=dict(visible=True, range=[0, blind_spots['abs_divergence'].max() * 1.1])
        ),
        height=600,
        width=600
    )
    
    return fig

def plot_consensus_strength(df, position=None, top_n=15):
    """Standard deviation between ranking sources"""
    
    # Filter by position if specified
    if position:
        df_filtered = df[df['position'] == position].copy()
        title_suffix = f" - {position} Only"
    else:
        df_filtered = df.copy()
        title_suffix = ""
    
    # Calculate ranking standard deviation
    df_filtered['rank_std'] = df_filtered[['espn_rank', 'adp_rank']].std(axis=1)
    
    # Get players with highest variance
    df_plot = df_filtered.nlargest(top_n, 'rank_std')
    
    # Create color based on which system ranks higher
    colors = ['#e74c3c' if divergence < 0 else '#3498db' 
              for divergence in df_plot['rank_divergence']]
    
    fig = go.Figure(data=go.Bar(
        x=df_plot['player_name'],
        y=df_plot['rank_std'],
        marker_color=colors,
        text=[f"ESPN: {int(espn)}<br>ADP: {int(adp)}" 
              for espn, adp in zip(df_plot['espn_rank'], df_plot['adp_rank'])],
        hovertemplate='Player: %{x}<br>Rank Std: %{y:.1f}<br>%{text}<extra></extra>'
    ))
    
    fig.update_layout(
        title=f'Consensus Weakness: High Ranking Variance{title_suffix}',
        xaxis_title='Player',
        yaxis_title='Ranking Standard Deviation',
        height=500,
        xaxis_tickangle=-45
    )
    
    # Add legend explanation
    fig.add_annotation(
        x=0.02, y=0.98,
        xref="paper", yref="paper",
        text="Red: ESPN ranks lower<br>Blue: ADP ranks lower",
        showarrow=False,
        bgcolor="white",
        bordercolor="black",
        borderwidth=1
    )
    
    return fig

# Create additional visualizations
radar_fig = plot_blind_spot_radar(enhanced_df, min_divergence=15)
if radar_fig:
    radar_fig.show()
    print("\n📊 Blind Spot Radar:")
    print("- Further from center = bigger divergence (more undervalued by ESPN)")
    print("- Larger markers = higher VBD value")
    print("- Green = high availability, Red = low availability")

consensus_fig = plot_consensus_strength(enhanced_df, top_n=12)
consensus_fig.show()

print("\n📊 Consensus Strength Analysis:")
print("- Higher bars = more disagreement between ranking sources")
print("- Red bars: ESPN ranks player lower than ADP")
print("- Blue bars: ADP ranks player lower than ESPN")
print("- High variance players require more careful evaluation")

NameError: name 'enhanced_df' is not defined

## 5. Additional Divergence Visualizations

Supplementary analysis functions for deeper blind spot detection.

In [ ]:
def create_divergence_summary_table(df, top_n=10):
    """Create summary table of biggest divergences with strategic advice"""
    
    # Get biggest positive and negative divergences
    positive_divergence = df[df['rank_divergence'] > 0].nlargest(top_n, 'rank_divergence')
    negative_divergence = df[df['rank_divergence'] < 0].nsmallest(top_n, 'rank_divergence')
    
    print("🚨 BIGGEST BLIND SPOTS - ESPN SIGNIFICANTLY UNDERVALUES:")
    print("=" * 80)
    for _, player in positive_divergence.iterrows():
        advice = "🎯 TARGET" if player['probability_available'] > 0.7 else "🚨 URGENT"
        print(f"{advice} {player['player_name']:<20} | ESPN:{player['espn_rank']:>3.0f} ADP:{player['adp_rank']:>3.0f} | "
              f"Divergence: +{player['rank_divergence']:>2.0f} | Avail: {player['probability_available']:.1%}")
    
    print("\n📈 BIGGEST OVERVALUES - ESPN SIGNIFICANTLY OVERVALUES:")
    print("=" * 80)
    for _, player in negative_divergence.iterrows():
        advice = "⏳ WAIT" if player['probability_available'] > 0.7 else "⚠️ AVOID"
        print(f"{advice} {player['player_name']:<20} | ESPN:{player['espn_rank']:>3.0f} ADP:{player['adp_rank']:>3.0f} | "
              f"Divergence: {player['rank_divergence']:>3.0f} | Avail: {player['probability_available']:.1%}")
    
    # Position-specific insights
    print(f"\n📊 POSITION BREAKDOWN:")
    print("=" * 50)
    for pos in ['RB', 'WR', 'QB', 'TE']:
        pos_df = df[df['position'] == pos]
        if len(pos_df) > 0:
            avg_divergence = pos_df['rank_divergence'].mean()
            undervalued = len(pos_df[pos_df['rank_divergence'] > 15])
            overvalued = len(pos_df[pos_df['rank_divergence'] < -15])
            print(f"{pos}: Avg Divergence: {avg_divergence:+.1f} | Undervalued: {undervalued} | Overvalued: {overvalued}")

# Create and display summary table
create_divergence_summary_table(enhanced_df, top_n=8)

print("\n" + "=" * 80)
print("🎯 STRATEGIC TAKEAWAYS:")
print("- Focus on 🎯 TARGET and 🚨 URGENT players (positive divergence)")
print("- Consider waiting on ⏳ WAIT players (negative divergence, high availability)")
print("- Avoid reaching for ⚠️ AVOID players (negative divergence, low availability)")
print("- The 80% ESPN weight may cause you to miss significant value")

## 4. Divergence Summary Table

Concrete examples of the biggest blind spots for immediate actionable insights.

In [ ]:
def plot_divergence_timeline(df, current_pick=1):
    """Bubble chart of divergence with VBD sizing"""
    
    # Focus on players likely to be available in first few rounds
    df_plot = df[df['adp_rank'] <= 100].copy()
    df_plot = df_plot.sort_values('adp_rank')
    
    fig = go.Figure()
    
    # Create hover text with player details
    hover_text = [
        f"{row['player_name']}<br>" +
        f"Position: {row['position']}<br>" +
        f"ESPN Rank: {row['espn_rank']}<br>" +
        f"ADP Rank: {row['adp_rank']}<br>" +
        f"VBD Score: {row['vbd_score']}<br>" +
        f"Availability: {row['probability_available']:.1%}"
        for _, row in df_plot.iterrows()
    ]
    
    fig.add_trace(go.Scatter(
        x=df_plot['adp_rank'],
        y=df_plot['rank_divergence'],
        mode='markers',
        marker=dict(
            size=np.maximum(df_plot['vbd_score']/3, 5),  # Minimum size 5
            color=df_plot['rank_divergence'],
            colorscale='RdBu',
            cmid=0,
            showscale=True,
            colorbar=dict(title="Rank Divergence<br>(ESPN - ADP)"),
            line=dict(width=1, color='black')
        ),
        text=hover_text,
        hovertemplate='%{text}<extra></extra>',
        name='Players'
    ))
    
    # Add reference lines
    fig.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    fig.add_hline(y=15, line_dash="dot", line_color="red", opacity=0.7, 
                  annotation_text="Significant Divergence")
    fig.add_hline(y=-15, line_dash="dot", line_color="blue", opacity=0.7,
                  annotation_text="Significant Divergence")
    
    # Add user's pick positions
    my_picks = [8, 17, 32, 41, 56, 65, 80, 89]
    for pick in my_picks[:4]:  # First 4 picks
        fig.add_vline(x=pick, line_dash="dash", line_color="green", opacity=0.6,
                      annotation_text=f"Pick {pick}")
    
    fig.update_layout(
        title=f'Divergence vs Draft Position (Current Pick: {current_pick})',
        xaxis_title='ADP Rank (Draft Position)',
        yaxis_title='Rank Divergence (ESPN - ADP)',
        height=600,
        width=1000,
        showlegend=False
    )
    
    return fig

# Create divergence timeline
timeline_fig = plot_divergence_timeline(enhanced_df, current_pick=1)
timeline_fig.show()

print("\n📊 Divergence Timeline:")
print("- X-axis: Where players are typically drafted (ADP)")
print("- Y-axis: How much ESPN disagrees with ADP")
print("- Bubble size: VBD value")
print("- Red bubbles above line: ESPN undervalues (potential steals)")
print("- Blue bubbles below line: ESPN overvalues (potential traps)")
print("- Green lines: Your draft picks")

## 3. Divergence Timeline

Bubble chart showing rank divergence across draft positions, with VBD scores determining bubble size. Helps identify when in the draft divergences are most significant.

In [ ]:
def plot_override_matrix(df, my_picks=[8, 17, 32, 41]):
    """Four quadrant plot for override decisions"""
    
    # Focus on players relevant to next few picks
    next_pick = my_picks[0]
    relevant_players = df[df['adp_rank'] <= next_pick + 20].copy()
    
    fig = go.Figure()
    
    # Add scatter plot
    fig.add_trace(go.Scatter(
        x=relevant_players['rank_divergence'],
        y=relevant_players['probability_available'],
        mode='markers+text',
        text=[name[:8] + "..." if len(name) > 8 else name 
              for name in relevant_players['player_name']],
        textposition="top center",
        marker=dict(
            size=np.maximum(relevant_players['vbd_score']/4, 6),
            color=relevant_players['vbd_score'],
            colorscale='Viridis',
            showscale=True,
            colorbar=dict(title="VBD Score"),
            line=dict(width=1, color='black')
        ),
        hovertemplate='Player: %{text}<br>Divergence: %{x}<br>Availability: %{y:.1%}<br>VBD: %{marker.color}<extra></extra>'
    ))
    
    # Add quadrant lines
    fig.add_hline(y=0.5, line_dash="dash", line_color="gray", opacity=0.5)
    fig.add_vline(x=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    # Add quadrant labels with strategic advice
    fig.add_annotation(
        x=-30, y=0.9,
        text="📈 ESPN OVERVALUED<br>🟢 LIKELY AVAILABLE<br>💡 Wait & Target Later",
        showarrow=False,
        bgcolor="lightgreen",
        bordercolor="green",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.add_annotation(
        x=30, y=0.9,
        text="📉 ESPN UNDERVALUED<br>🟢 LIKELY AVAILABLE<br>🎯 Hidden Gems",
        showarrow=False,
        bgcolor="lightblue",
        bordercolor="blue",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.add_annotation(
        x=-30, y=0.1,
        text="📈 ESPN OVERVALUED<br>🔴 WILL BE GONE<br>⚠️ Avoid - Others Overvaluing",
        showarrow=False,
        bgcolor="lightyellow",
        bordercolor="orange",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.add_annotation(
        x=30, y=0.1,
        text="📉 ESPN UNDERVALUED<br>🔴 WILL BE GONE<br>🚨 Draft Now or Miss Out",
        showarrow=False,
        bgcolor="lightcoral",
        bordercolor="red",
        borderwidth=1,
        font=dict(size=10)
    )
    
    fig.update_layout(
        title=f'Strategic Override Decision Matrix (Next Pick: {next_pick})',
        xaxis_title='Rank Divergence (ESPN - ADP)',
        yaxis_title='Probability Available at Your Next Pick',
        height=700,
        width=1000,
        showlegend=False
    )
    
    return fig

# Create strategic override matrix
override_fig = plot_override_matrix(enhanced_df, my_picks=[8, 17, 32, 41])
override_fig.show()

print("\n📊 Strategic Override Matrix:")
print("🎯 Top Right (Blue): Hidden gems - ESPN undervalues, still available")
print("🚨 Bottom Right (Red): Urgent picks - ESPN undervalues, will be gone")  
print("🟢 Top Left (Green): Wait targets - ESPN overvalues, will be available")
print("⚠️  Bottom Left (Yellow): Avoid - ESPN overvalues, others will take anyway")

## 2. Strategic Override Matrix

Four-quadrant analysis for override decisions. Helps identify when to override the 80/20 system based on divergence and availability.

In [ ]:
def plot_queue_jump_heatmap(df, upcoming_picks=[8, 17, 32]):
    """Shows players likely to jump queue at your picks"""
    
    # Create pick windows around your upcoming picks
    pick_windows = [(max(1, p-5), p+5) for p in upcoming_picks[:3]]
    window_labels = [f"Pick {p} Window" for p in upcoming_picks[:3]]
    
    heatmap_data = []
    player_names = []
    
    for start, end in pick_windows:
        # Get players in this ADP range
        window_df = df[(df['adp_rank'] >= start) & (df['adp_rank'] <= end)]
        window_df = window_df.sort_values('adp_rank').head(10)  # Top 10 in window
        
        # Store divergence values and player names
        divergences = window_df['rank_divergence'].values
        # Pad with zeros if less than 10 players
        if len(divergences) < 10:
            divergences = np.concatenate([divergences, [0] * (10 - len(divergences))])
        
        heatmap_data.append(divergences)
        
        if len(player_names) == 0:  # Only get names once
            names = window_df['player_name'].tolist()
            names.extend([''] * (10 - len(names)))  # Pad with empty strings
            player_names = names[:10]
    
    fig = go.Figure(data=go.Heatmap(
        z=heatmap_data,
        x=player_names,
        y=window_labels,
        colorscale='RdBu',
        zmid=0,
        colorbar=dict(title="Rank Divergence<br>(ESPN - ADP)"),
        hovetemplate='Window: %{y}<br>Player: %{x}<br>Divergence: %{z}<extra></extra>'
    ))
    
    fig.update_layout(
        title='Queue Jump Alert: Rank Divergence at Your Draft Picks',
        xaxis_title='Players (sorted by ADP in first window)',
        yaxis_title='Draft Pick Windows',
        height=400,
        width=1000
    )
    
    return fig

# Create queue jump heatmap
jump_fig = plot_queue_jump_heatmap(enhanced_df, [8, 17, 32])
jump_fig.show()

print("\n📊 Queue Jump Analysis:")
print("- Red (negative): ESPN ranks much lower than ADP - likely to be picked earlier")
print("- Blue (positive): ADP ranks much lower than ESPN - potential value")
print("- Focus on red players in your pick windows - they may jump the queue")

## 1. Queue Jump Alert Heatmap

Shows players likely to jump ahead of their ADP position at your specific draft picks. Red areas indicate ESPN ranks players much higher than ADP suggests.

In [ ]:
# Strategic Divergence Functions

def add_divergence_metrics(df):
    """Add divergence calculations to existing dataframe"""
    df = df.copy()
    df['rank_divergence'] = df['espn_rank'] - df['adp_rank']
    df['abs_divergence'] = abs(df['rank_divergence'])
    
    # Add availability probabilities for analysis
    my_picks = [8, 17, 32, 41, 56, 65, 80, 89]
    current_pick = 1
    next_pick = min([p for p in my_picks if p > current_pick])
    picks_until = next_pick - current_pick
    
    availability_probs = []
    for _, player in df.iterrows():
        prob_gone = probability_gone_before_next_pick(df, player['player_name'], picks_until)
        availability_probs.append(1 - prob_gone)
    
    df['probability_available'] = availability_probs
    df['vbd_score'] = df['salary_value']  # Use salary_value as VBD proxy
    
    return df

# Apply divergence metrics to our data
enhanced_df = add_divergence_metrics(ranking_df)
print("✅ Enhanced dataframe with divergence metrics")
print(f"Divergence range: {enhanced_df['rank_divergence'].min():.1f} to {enhanced_df['rank_divergence'].max():.1f}")
print(f"Players with significant negative divergence (ESPN overvalued): {len(enhanced_df[enhanced_df['rank_divergence'] < -15])}")
print(f"Players with significant positive divergence (ADP overvalued): {len(enhanced_df[enhanced_df['rank_divergence'] > 15])}")

# Strategic Divergence Analysis - Blind Spot Detection

This section implements visualizations to identify strategic blind spots in the 80% ESPN + 20% ADP system by analyzing ranking divergences and highlighting opportunities where the weighting may mislead.

# Statistical Visualization Functions for Fantasy Football Draft Probability System

This notebook implements comprehensive statistical visualizations for analyzing the 80% ESPN + 20% ADP probability system with discrete survival calculations.

## Key Visualizations:
1. **Softmax Temperature Analysis** - How τ affects probability concentration
2. **80/20 Weighting Impact** - Compare ESPN vs ADP vs blended probabilities
3. **Discrete Survival Waterfall** - Step-by-step survival probability
4. **Normal vs Exponential Comparison** - Old vs new distribution methods
5. **Probability Evolution** - How availability changes through draft
6. **Decision Zone Mapping** - VBD value vs availability zones
7. **Sensitivity Analysis** - Parameter robustness testing

In [ ]:
# Required imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import norm
import sys
import os

# Add parent directory to path to import functions from main notebook
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

print("✅ Imports loaded successfully")

✅ Imports loaded successfully


In [ ]:
# Load core functions from main notebook
def load_and_merge_ranking_data():
    """Load ESPN rankings and external ADP data, merge them for probability calculations"""
    
    # Load ESPN data (80% weight)
    espn_df = pd.read_csv('../data/espn_projections_20250814.csv')
    
    # Load FantasyPros ADP data (20% weight)  
    adp_df = pd.read_csv('../data/fantasypros_adp_20250815.csv')
    
    print(f"ESPN data: {len(espn_df)} players")
    print(f"ADP data: {len(adp_df)} players")
    
    # Standardize player names for matching
    def clean_name(name):
        return str(name).strip().lower().replace("'", "").replace(".", "")
    
    espn_df['clean_name'] = espn_df['player_name'].apply(clean_name)
    adp_df['clean_name'] = adp_df['PLAYER'].apply(clean_name)
    
    # Merge ESPN with ADP data
    merged_df = espn_df.merge(
        adp_df[['clean_name', 'RANK', 'ADP']], 
        on='clean_name', 
        how='left'
    )
    
    # Fill missing ADP ranks with ESPN rank + some penalty
    merged_df['adp_rank'] = merged_df['RANK'].fillna(merged_df['overall_rank'] + 50)
    merged_df['espn_rank'] = merged_df['overall_rank']
    
    print(f"Merged dataset: {len(merged_df)} players")
    print(f"Players with ADP data: {len(merged_df[merged_df['RANK'].notna()])} players")
    
    return merged_df[['overall_rank', 'position', 'position_rank', 'player_name', 'team', 'salary_value', 'bye_week', 'espn_rank', 'adp_rank']]

def compute_softmax_scores(rank_series, tau=5.0):
    """Convert ranks to softmax scores with temperature tau (lower rank = higher score)"""
    scores = np.exp(-rank_series / tau)
    return scores

def compute_pick_probabilities(available_df, espn_weight=0.8, adp_weight=0.2, tau_espn=5.0, tau_adp=5.0):
    """
    Compute per-player probability of being picked next using weighted softmax over ESPN and ADP ranks
    """
    if len(available_df) == 0:
        return pd.Series(dtype=float)
    
    # Softmax scores for ESPN and ADP ranks
    espn_scores = compute_softmax_scores(available_df['espn_rank'], tau_espn)
    adp_scores = compute_softmax_scores(available_df['adp_rank'], tau_adp)
    
    # Weighted combination (80% ESPN, 20% ADP)
    combined_scores = espn_weight * espn_scores + adp_weight * adp_scores
    
    # Normalize to sum to 1 across available players
    probs = combined_scores / combined_scores.sum()
    return probs

def probability_gone_before_next_pick(available_df, player_name, picks_until_next_turn):
    """
    Compute probability a player is gone before your next turn using discrete survival calculation
    """
    if picks_until_next_turn <= 0:
        return 0.0
    
    survival_prob = 1.0
    current_available = available_df.copy()
    
    # Simulate each pick until our next turn
    for pick_step in range(picks_until_next_turn):
        if len(current_available) == 0:
            break
            
        # Get probabilities for current available players
        pick_probs = compute_pick_probabilities(current_available)
        
        # Find probability our target player gets picked this round
        player_mask = current_available['player_name'] == player_name
        if not player_mask.any():
            # Player already gone
            break
            
        p_pick_now = pick_probs[player_mask].iloc[0] if player_mask.any() else 0.0
        
        # Update survival probability
        survival_prob *= (1 - p_pick_now)
        
        # For simulation: remove the most likely pick (simplified approach)
        most_likely_idx = pick_probs.idxmax()
        current_available = current_available.drop(most_likely_idx)
    
    prob_gone = 1 - survival_prob
    return min(1.0, max(0.0, prob_gone))

# Load data for examples
ranking_df = load_and_merge_ranking_data()
print("\n✅ Core functions and data loaded")

ESPN data: 300 players
ADP data: 347 players
Merged dataset: 300 players
Players with ADP data: 252 players

✅ Core functions and data loaded


## 1. Softmax Temperature Analysis

This visualization shows how the temperature parameter τ (tau) affects the concentration of pick probabilities. Higher τ values spread probabilities more evenly, while lower τ values concentrate probability on top-ranked players.

In [ ]:
def plot_temperature_impact(ranks, tau_values=[1, 3, 5, 10]):
    """Show how τ affects probability concentration"""
    fig = go.Figure()
    
    for tau in tau_values:
        scores = np.exp(-ranks / tau)
        probs = scores / scores.sum()
        fig.add_trace(go.Scatter(
            x=ranks, 
            y=probs, 
            name=f'τ={tau}',
            mode='lines+markers'
        ))
    
    fig.update_layout(
        title='Softmax Temperature Effect on Pick Probabilities',
        xaxis_title='Player Rank', 
        yaxis_title='Pick Probability',
        height=500,
        showlegend=True
    )
    
    return fig

# Example: Show temperature effect for ranks 1-20
example_ranks = np.arange(1, 21)
temp_fig = plot_temperature_impact(example_ranks)
temp_fig.show()

print("\n📊 Temperature Analysis:")
print("- Lower τ (tau=1): Highly concentrated on top players")
print("- Higher τ (tau=10): More evenly distributed probabilities")
print("- Our system uses τ=5 as a balanced middle ground")


📊 Temperature Analysis:
- Lower τ (tau=1): Highly concentrated on top players
- Higher τ (tau=10): More evenly distributed probabilities
- Our system uses τ=5 as a balanced middle ground


## 2. 80/20 Weighting Impact

Compare how different weighting schemes (ESPN only, ADP only, or 80/20 blend) affect individual player pick probabilities.

## Complete Analysis Summary

This notebook provides comprehensive statistical visualizations for understanding and validating the fantasy football draft probability system:

### Core Statistical Analysis:
1. **Temperature Analysis**: Validates softmax parameter choice
2. **Weighting Impact**: Shows effect of 80/20 ESPN/ADP blend
3. **Survival Waterfall**: Illustrates discrete probability calculation
4. **Distribution Comparison**: Validates exponential vs normal approach
5. **Timeline Evolution**: Shows probability changes through draft
6. **Decision Zones**: Strategic guidance visualization
7. **Sensitivity Analysis**: Parameter robustness testing
8. **Comparative Analysis**: Ranking source validation

### Strategic Divergence Analysis:
9. **Queue Jump Alerts**: Heatmap of rank divergence at your picks
10. **Override Matrix**: Four-quadrant strategic decision framework
11. **Divergence Timeline**: Bubble chart of divergence by draft position
12. **Divergence Summary**: Actionable lists of targets and avoids
13. **Blind Spot Radar**: Radial view of ESPN undervalued players
14. **Consensus Weakness**: High variance players requiring careful evaluation

These visualizations help validate the probability system and provide insights for draft strategy optimization, while identifying critical blind spots where the 80% ESPN weighting may mislead.

## 3. Discrete Survival Waterfall

Visualizes step-by-step survival probability as each pick occurs before your next turn.

In [ ]:
def plot_survival_waterfall(df, player_name, picks_until=5):
    """Step-by-step survival probability visualization"""
    survival = 1.0
    steps = [survival]
    current_df = df.copy()
    
    for pick in range(picks_until):
        if len(current_df) == 0:
            break
            
        pick_probs = compute_pick_probabilities(current_df)
        
        # Find our player's probability of being picked this round
        player_mask = current_df['player_name'] == player_name
        if not player_mask.any():
            break
            
        p_picked = pick_probs[player_mask].iloc[0]
        survival *= (1 - p_picked)
        steps.append(survival)
        
        # Remove most likely player for next iteration
        most_likely_idx = pick_probs.idxmax()
        current_df = current_df.drop(most_likely_idx)
    
    # Create waterfall chart
    pick_labels = ['Start'] + [f'Pick {i+1}' for i in range(len(steps)-1)]
    
    fig = go.Figure(go.Waterfall(
        name="Survival Probability",
        orientation="v",
        measure=["absolute"] + ["relative"] * (len(steps)-1),
        x=pick_labels,
        y=[steps[0]] + [steps[i] - steps[i-1] for i in range(1, len(steps))],
        connector={"line":{"color":"rgb(63, 63, 63)"}},
    ))
    
    fig.update_layout(
        title=f"{player_name} Survival Probability Waterfall",
        yaxis_title="Survival Probability",
        height=500
    )
    
    return fig

# Example with a mid-tier player
example_player_mid = ranking_df.iloc[15]['player_name']
waterfall_fig = plot_survival_waterfall(ranking_df, example_player_mid, picks_until=7)
waterfall_fig.show()

print(f"\n📊 Survival Waterfall for {example_player_mid}:")
print("- Shows cumulative effect of each pick reducing survival probability")
print("- Discrete simulation more realistic than normal distribution assumption")


📊 Survival Waterfall for Drake London:
- Shows cumulative effect of each pick reducing survival probability
- Discrete simulation more realistic than normal distribution assumption


## 4. Normal vs Exponential Distribution Comparison

Compares the old normal distribution approach with the new exponential decay (softmax) approach.

In [ ]:
def plot_distribution_comparison(rank_range=30):
    """Overlay normal (old) vs exponential (new) decay"""
    ranks = np.arange(1, rank_range + 1)
    
    # Old: normal distribution (centered around rank 15 with std=3)
    normal_probs = norm.pdf(ranks, loc=15, scale=3)
    normal_probs = normal_probs / normal_probs.sum()  # Normalize
    
    # New: exponential decay (softmax with tau=5)
    exp_probs = np.exp(-ranks / 5)
    exp_probs = exp_probs / exp_probs.sum()  # Normalize
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=ranks, 
        y=normal_probs, 
        name='Normal Distribution (Old)',
        line=dict(color='#e74c3c', width=3)
    ))
    
    fig.add_trace(go.Scatter(
        x=ranks, 
        y=exp_probs, 
        name='Exponential Decay (New)',
        line=dict(color='#2ecc71', width=3)
    ))
    
    fig.update_layout(
        title='Distribution Comparison: Normal vs Exponential Decay',
        xaxis_title='Player Rank',
        yaxis_title='Pick Probability Density',
        height=500,
        showlegend=True
    )
    
    return fig

dist_fig = plot_distribution_comparison()
dist_fig.show()

print("\n📊 Distribution Comparison:")
print("- Normal (Old): Symmetric, peaks in middle ranks")
print("- Exponential (New): Heavy concentration on top ranks, realistic decay")
print("- New approach better reflects actual draft behavior")


📊 Distribution Comparison:
- Normal (Old): Symmetric, peaks in middle ranks
- Exponential (New): Heavy concentration on top ranks, realistic decay
- New approach better reflects actual draft behavior


## 5. Probability Evolution Timeline

Shows how a player's availability probability changes as the draft progresses toward your pick positions.

In [ ]:
def plot_probability_timeline(df, player_name, my_picks):
    """Show how availability probability changes through picks"""
    probs = []
    pick_numbers = []
    
    for current_pick in range(1, max(my_picks) + 1):
        # Find next pick after current pick
        future_picks = [p for p in my_picks if p > current_pick]
        if future_picks:
            next_pick = min(future_picks)
            picks_until = next_pick - current_pick
            
            # Calculate survival probability
            prob_gone = probability_gone_before_next_pick(df, player_name, picks_until)
            prob_available = 1 - prob_gone
            
            probs.append(prob_available)
            pick_numbers.append(current_pick)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=pick_numbers, 
        y=probs,
        mode='lines+markers',
        name=f'{player_name} Availability',
        line=dict(width=3)
    ))
    
    # Add vertical lines for user's picks
    for pick in my_picks:
        if pick <= max(pick_numbers):
            fig.add_vline(
                x=pick, 
                line_dash="dash", 
                line_color="red",
                opacity=0.7,
                annotation_text=f"Your Pick {pick}"
            )
    
    fig.update_layout(
        title=f'{player_name} Availability Probability Timeline',
        xaxis_title='Current Draft Pick',
        yaxis_title='Probability Available at Next Your Pick',
        height=500,
        yaxis=dict(range=[0, 1])
    )
    
    return fig

# Example with your draft picks
my_picks = [8, 17, 32, 41, 56, 65, 80, 89]
example_player_timeline = ranking_df.iloc[10]['player_name']

timeline_fig = plot_probability_timeline(ranking_df, example_player_timeline, my_picks)
timeline_fig.show()

print(f"\n📊 Timeline for {example_player_timeline}:")
print("- Shows decreasing availability as draft progresses")
print("- Red lines mark your draft positions")
print("- Helps identify optimal timing for targeting players")


📊 Timeline for Derrick Henry:
- Shows decreasing availability as draft progresses
- Red lines mark your draft positions
- Helps identify optimal timing for targeting players


## 6. Decision Zone Mapping

2D scatter plot showing VBD value vs availability probability, with color-coded decision zones.

In [ ]:
def plot_decision_zones(df, vbd_col='salary_value', picks_until=7):
    """2D scatter: VBD value vs availability probability"""
    
    # Calculate availability probabilities for all players
    availability_probs = []
    for _, player in df.iterrows():
        prob_gone = probability_gone_before_next_pick(df, player['player_name'], picks_until)
        availability_probs.append(1 - prob_gone)
    
    plot_df = df.copy()
    plot_df['availability_prob'] = availability_probs
    
    # Create decision zones
    plot_df['zone'] = pd.cut(
        plot_df['availability_prob'], 
        bins=[0, 0.3, 0.8, 1.0],
        labels=['REACH', 'DRAFT NOW', 'SAFE']
    )
    
    # Filter to players with meaningful VBD scores
    plot_df = plot_df[plot_df[vbd_col] > 0]
    
    fig = px.scatter(
        plot_df, 
        x='availability_prob', 
        y=vbd_col,
        color='zone', 
        hover_data=['player_name', 'position', 'team'],
        color_discrete_map={
            'REACH': '#e74c3c',
            'DRAFT NOW': '#f39c12', 
            'SAFE': '#2ecc71'
        },
        title='Decision Zone Mapping - VBD Value vs Availability'
    )
    
    # Add zone boundary lines
    fig.add_vline(x=0.3, line_dash="dash", line_color="gray", annotation_text="Reach Threshold")
    fig.add_vline(x=0.8, line_dash="dash", line_color="gray", annotation_text="Safe Threshold")
    
    fig.update_layout(
        xaxis_title='Availability Probability at Next Pick',
        yaxis_title='VBD Score',
        height=600,
        width=900
    )
    
    return fig

# Create decision zones plot
decision_fig = plot_decision_zones(ranking_df, picks_until=7)
decision_fig.show()

print("\n📊 Decision Zone Mapping:")
print("- Green (SAFE): >80% available, can wait")
print("- Yellow (DRAFT NOW): 30-80% available, risky to wait")
print("- Red (REACH): <30% available, must draft now")
print("- High VBD + Low availability = priority targets")


📊 Decision Zone Mapping:
- Green (SAFE): >80% available, can wait
- Yellow (DRAFT NOW): 30-80% available, risky to wait
- Red (REACH): <30% available, must draft now
- High VBD + Low availability = priority targets


## 7. Sensitivity Analysis

Tests how robust the probability calculations are to changes in key parameters like temperature (τ).

In [ ]:
def plot_sensitivity(df, param_name='tau', param_range=[3, 5, 7, 10]):
    """Test robustness to parameter changes"""
    top_5_players = df.head(5)['player_name'].tolist()
    
    fig = go.Figure()
    
    for i, player in enumerate(top_5_players):
        player_probs = []
        
        for param_val in param_range:
            if param_name == 'tau':
                # Calculate probabilities with different tau values
                probs = compute_pick_probabilities(df, tau_espn=param_val, tau_adp=param_val)
                player_idx = df.index[df['player_name'] == player][0]
                player_prob = probs.loc[player_idx]
                player_probs.append(player_prob)
        
        fig.add_trace(go.Scatter(
            x=param_range, 
            y=player_probs,
            name=player,
            mode='lines+markers'
        ))
    
    fig.update_layout(
        title=f'Sensitivity Analysis - Parameter: {param_name}',
        xaxis_title=f'{param_name} Value',
        yaxis_title='Pick Probability',
        height=500
    )
    
    return fig

# Test sensitivity to tau parameter
sensitivity_fig = plot_sensitivity(ranking_df, param_name='tau', param_range=[2, 4, 6, 8, 10])
sensitivity_fig.show()

print("\n📊 Sensitivity Analysis:")
print("- Shows how parameter changes affect top players' probabilities")
print("- Relatively stable results indicate robust system")
print("- Higher tau spreads probabilities more evenly")


📊 Sensitivity Analysis:
- Shows how parameter changes affect top players' probabilities
- Relatively stable results indicate robust system
- Higher tau spreads probabilities more evenly


## 8. Comparative Analysis Functions

Additional functions for comparing different aspects of the probability system.

In [ ]:
def compare_ranking_sources(df, top_n=10):
    """Compare ESPN vs ADP rankings for top players"""
    top_players = df.head(top_n)
    
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=top_players['espn_rank'],
        y=top_players['adp_rank'],
        mode='markers+text',
        text=top_players['player_name'],
        textposition="top center",
        marker=dict(size=10, color='blue'),
        name='Players'
    ))
    
    # Add diagonal line for perfect correlation
    max_rank = max(top_players['espn_rank'].max(), top_players['adp_rank'].max())
    fig.add_trace(go.Scatter(
        x=[1, max_rank],
        y=[1, max_rank],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='Perfect Correlation'
    ))
    
    fig.update_layout(
        title='ESPN vs ADP Rankings Comparison',
        xaxis_title='ESPN Rank',
        yaxis_title='ADP Rank',
        height=600,
        width=600
    )
    
    return fig

def plot_probability_distribution(df, picks_until=7):
    """Show distribution of availability probabilities across all players"""
    probs = []
    for _, player in df.iterrows():
        prob_gone = probability_gone_before_next_pick(df, player['player_name'], picks_until)
        probs.append(1 - prob_gone)
    
    fig = go.Figure(data=[go.Histogram(
        x=probs,
        nbinsx=20,
        marker_color='lightblue',
        opacity=0.7
    )])
    
    fig.update_layout(
        title=f'Distribution of Availability Probabilities ({picks_until} picks until next turn)',
        xaxis_title='Availability Probability',
        yaxis_title='Number of Players',
        height=400
    )
    
    return fig

# Create comparison plots
ranking_comparison = compare_ranking_sources(ranking_df)
ranking_comparison.show()

prob_distribution = plot_probability_distribution(ranking_df)
prob_distribution.show()

print("\n📊 Additional Analysis:")
print("- Ranking comparison shows agreement/disagreement between sources")
print("- Probability distribution reveals how spread out availability estimates are")
print("- Points far from diagonal line indicate ranking source disagreement")


📊 Additional Analysis:
- Ranking comparison shows agreement/disagreement between sources
- Probability distribution reveals how spread out availability estimates are
- Points far from diagonal line indicate ranking source disagreement


## Summary

This notebook provides comprehensive statistical visualizations for understanding and validating the fantasy football draft probability system:

1. **Temperature Analysis**: Validates softmax parameter choice
2. **Weighting Impact**: Shows effect of 80/20 ESPN/ADP blend
3. **Survival Waterfall**: Illustrates discrete probability calculation
4. **Distribution Comparison**: Validates exponential vs normal approach
5. **Timeline Evolution**: Shows probability changes through draft
6. **Decision Zones**: Strategic guidance visualization
7. **Sensitivity Analysis**: Parameter robustness testing
8. **Comparative Analysis**: Ranking source validation

These visualizations help validate the probability system and provide insights for draft strategy optimization.